<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_judge_answer_zeroshot_MoritzLaurer_mDeBERTa_v3_base_mnli_xnli_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#한국 버블 커뮤니티 오픈톡방 대화의 질문에 대한 답변을 선별하는 코드(zero-shot text classification)(MoritzLaurer/mDeBERTa-v3-base-mnli-xnli)(로컬)

In [ ]:
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import urllib.request
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline, pipeline
!pip install datasets

In [ ]:
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, load_dataset
import torch
from google.colab import drive
from sklearn.model_selection import StratifiedKFold, train_test_split
drive.mount('/content/drive')

In [ ]:
#모델 불러오기
HUGGINGFACE_MODEL_PATH = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"

classifier = pipeline("zero-shot-classification", model = HUGGINGFACE_MODEL_PATH)

In [ ]:
#모델 실험
sequence_to_classify = "Angela Merkel ist eine Politikerin in Deutschland und Vorsitzende der CDU"
candidate_labels = ["politics", "economy", "entertainment", "environment"]
output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
print(output)
print(output['labels'][0])

In [ ]:
#훈련 데이터 불러오기
dataset_path = '/content/drive/My Drive/KakaoTalkChats-1.txt'

In [ ]:
#카카오톡 데이터 불러오기
file_path = '/content/drive/My Drive/judge_question_result.csv'

In [ ]:
#카카오톡 대화내용을 데이터프레임으로 받기
df = pd.read_csv(file_path)

#질문 딕셔너리, 답변 목록 리스트(이중 리스트) 생성
questions, answer_lists = {}, []

#질문으로 판별된 텍스트를 새 데이터프레임으로 생성
df_question = df[df['label'] == 'question']

print(df_question)
#새 데이터프레임의 index, text를 question 딕셔너리에 저장
questions = {text : index for (index, text) in zip(df_question.index, df_question['text'])}
print(questions)

In [ ]:
# Ensure the classifier is using GPU
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available, otherwise CPU
print(device)
classifier = pipeline('zero-shot-classification', model=HUGGINGFACE_MODEL_PATH, device=device, batch_size = 64)

In [ ]:
#전체 질문에 대해서 각 질문 이후 30개의 텍스트에 대해서 판별, 이때 특정 텍스트가 답변이 맞으면 답변 리스트에 추가한 뒤 다음 텍스트 판별

# Pre-fetch the necessary DataFrame data to minimize access within loops
question_texts = {question: df.iloc[questions[question] + 1 : questions[question] + 6]['text'].tolist() for question in questions.keys()}

answer_lists = []
i = 0
for question, texts in tqdm(question_texts.items(), desc="Processing questions"):
    answer = []
    for text in texts: #나중에 숫자 커지면 texts 대신 tqdm(texts, desc="Classifying texts", leave=False)
        sequence_to_classify = question + ' '.join(answer + [text])
        #동적 레이블
        candidate_labels = ["continuing answer", "not continuing answer"] if answer else ["question-answer pair", "not question-answer pair"]

        # Process the classification in one step
        output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
        expected_label = "continuing answer" if answer else "question-answer pair"
        if output['labels'][0] == expected_label:
            answer.append(text)
    answer_lists.append(answer) if len(answer) != 0 else answer_lists.append(['No answer'])

In [ ]:
print(answer_lists, end = '\n')

In [ ]:
# #데이터프레임 가공
# df_question.rename(columns = {'text' : 'question'})
# df_question['answer'] = answer_lists

data = {question : answer for question, answer in zip(questions.keys(), answer_lists)}
print(data)
data_result= [{'question' : key, 'answer' : value} for key, value in data.items()]
df_result = pd.DataFrame(data = data_result, columns = ['question', 'answer'])
print(df_result)

In [ ]:
#.csv 파일로 google drive에 저장
csv_file_path = '/content/drive/My Drive/judge_answer_result_MoritzLaurer_mDeBERTa-v3-base-mnli-xnli.csv'

df_result.to_csv(csv_file_path)